In [ ]:
# XGBoost Example
# I acknowledge "Sreenivas Bhattiprolu" ( https://github.com/bnsreenu/python_for_microscopists),
# the based code for XGBoost has been taken from his git 
# I added two main things:
#     -The posibility to load several Images as training
#     -A set of features including: gamma constrasting, H&E Stainnig, range color
# Code is structed to be used with randomForest, SVM or XGBoost
import glob
import pickle
path = "C:/ImageFolder/*.*"
fileArra=[]
for file in glob.glob(path):
    print(file) 
    fileArra.append(file)

In [2]:
from skimage import data
from skimage.color import (separate_stains, combine_stains,hdx_from_rgb, rgb_from_hdx,rgb_from_fgx,fgx_from_rgb)
from skimage.color import (rgb_from_bex,bex_from_rgb)
import cv2 as cv
import numpy as np
import functools
import numpy as np
from warnings import warn
from scipy import linalg
from skimage.util import dtype, dtype_limits
from skimage import exposure
import xgboost as xgb


def _prepare_colorarray(arr, force_copy=False):
    """Check the shape of the array and convert it to
    floating point representation.
    """
    arr = np.asanyarray(arr)

    if arr.shape[-1] != 3:
        raise ValueError("Input array must have a shape == (..., 3)), "
                         f"got {arr.shape}")

    return dtype.img_as_float(arr, force_copy=force_copy)

def separate_StainsS(ihc,conv_matrix):
    rgb = _prepare_colorarray(ihc, force_copy=True)
    np.maximum(rgb, 1E-6, out=rgb)  # avoiding log artifacts
    log_adjust = np.log(1E-6)  # used to compensate the sum above
    stains = (np.log(rgb) / log_adjust) @ conv_matrix
    np.maximum(stains, 0, out=stains)
    return stains

def combine_stainsA(stains, conv_matrix):
    log_adjust = -np.log(1E-6)
    log_rgb = -(stains * log_adjust) @ conv_matrix
    rgb = np.exp(log_rgb)
    return np.clip(rgb, a_min=0, a_max=1)

def histCalculation(ihc):
    ihc = cv.cvtColor(ihc, cv.COLOR_BGR2RGB)
    conv= bex_from_rgb
    stains= separate_StainsS(ihc,conv)
    ihc_hdx= stains
    # Create an RGB image for each of the stains
    null = np.zeros_like(ihc_hdx[:, :, 0])
    conv_Inv=rgb_from_bex 
    ihc_e = combine_stainsA(np.stack((null,ihc_hdx[:, :, 1], null), axis=-1), conv_Inv)
    newImag= np.array(ihc_e*255, dtype="uint8")
    outImagHist=cv.cvtColor(newImag, cv.COLOR_BGR2GRAY)
    return outImagHist

# Range Threshold

def rangeFilter(imHSV):
    h1=round(190/360*180)
    h2=round(345/360*180)
    s1=round(10/100*255)
    s2=round(100/100*180)
    v1=round(20/100*255)
    v2=255
    print(h1,s1,v1,"--",h2,s2,v2)
    lower_red = np.array([h1,s1,v1])
    upper_red = np.array([h2,s2,v2])
    mask1 = cv.inRange(imHSV, lower_red, upper_red)
    return mask1

def gammaContrastFilter(imRGB,imHSV):
    gamma_corrected = exposure.adjust_gamma(imRGB, 10)
    imgInput= gamma_corrected
    imgGray = cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY) 
    imagem = cv2.bitwise_not(imgGray)
    a,thinned= cv.threshold(imagem, 100, 255, cv.THRESH_BINARY)
    lower_red = np.array([50,20,1])
    upper_red = np.array([80,255,240])
    mask = cv.inRange(imHSV, lower_red, upper_red)
    mask=cv.dilate(mask, None)
    mask=cv.dilate(mask, None)
    image3 = cv.subtract(thinned, mask)
    return image3

In [3]:
import numpy as np
import cv2
import pandas as pd
imagArra=[]
imagGreyArra=[]
imgHueFull= np.array([])
imgSaturationFull=np.array([])
imgValueFull=np.array([])
imgGreyFull=np.array([])
imgHistFull=np.array([])
imgRangFull=np.array([])
imgGammFull=np.array([])
for data in fileArra:
    imgInput = cv2.imread(data)
    img = cv2.cvtColor(imgInput, cv2.COLOR_BGR2HSV) 
    img2= cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY)
    imagArra.append(img)
    imagGreyArra.append(img2)
    img_hue= img[:,:,0]
    img_saturation= img[:,:,1]
    img_value= img[:,:,2]
    img2_hue = img_hue.reshape(-1)
    img2_saturation = img_saturation.reshape(-1)
    img2_value = img_value.reshape(-1)
    img2_grey = img2.reshape(-1)
    img_hist= histCalculation(imgInput)
    img2_hist= img_hist.reshape(-1)
    img_Range= rangeFilter(img)
    img2_Range=  img_Range.reshape(-1)
    img_Gamma= gammaContrastFilter(imgInput,img)
    img2_Gamma= img_Gamma.reshape(-1)
    imgHueFull=np.concatenate((imgHueFull,img2_hue),axis=None)
    imgSaturationFull=np.concatenate((imgSaturationFull,img2_saturation),axis=None)
    imgValueFull=np.concatenate((imgValueFull,img2_value),axis=None)
    imgGreyFull=np.concatenate((imgGreyFull,img2_grey),axis=None)
    imgHistFull=np.concatenate((imgHistFull,img2_hist),axis=None)
    imgRangFull=np.concatenate((imgRangFull,img2_Range),axis=None)
    imgGammFull=np.concatenate((imgGammFull,img2_Gamma),axis=None)

95 26 51 -- 172 180 255
95 26 51 -- 172 180 255
95 26 51 -- 172 180 255
95 26 51 -- 172 180 255
95 26 51 -- 172 180 255
95 26 51 -- 172 180 255
95 26 51 -- 172 180 255
95 26 51 -- 172 180 255


In [4]:
#Multiple images can be used for training. For that, you need to concatenate the data
#Save original image pixels into a data frame. This is our Feature #1.

In [5]:
df = pd.DataFrame()
df['Original Image Hue'] = imgHueFull

In [6]:
df['Original Image Saturation'] = imgSaturationFull

In [7]:
df['Original Image Value'] = imgValueFull

In [8]:
df['Original Image'] = imgGreyFull

In [9]:
df['Original Image Hist'] = imgHistFull

In [10]:
df['Original Image Range'] = imgRangFull

In [11]:
df['Original Image Gamma'] = imgGammFull

In [12]:
df

,Original Image Hue,Original Image Saturation,Original Image Value,Original Image,Original Image Hist,Original Image Range,Original Image Gamma
0,62.0,14.0,255.0,249.0,255.0,0.0,0.0
1,62.0,16.0,255.0,249.0,255.0,0.0,0.0
2,62.0,15.0,255.0,249.0,255.0,0.0,0.0
3,62.0,14.0,255.0,249.0,255.0,0.0,0.0
4,62.0,14.0,251.0,245.0,255.0,0.0,0.0
...,...,...,...,...,...,...,...
1290370,35.0,11.0,249.0,247.0,251.0,0.0,0.0
1290371,51.0,7.0,243.0,241.0,251.0,0.0,255.0
1290372,48.0,5.0,255.0,254.0,255.0,0.0,0.0
1290373,36.0,5.0,255.0,254.0,254.0,0.0,0.0


In [13]:
from scipy import ndimage as nd
from skimage.filters import roberts, sobel, scharr, prewitt

edges1Full= np.array([])
edge_roberts1Full = np.array([])
edge_sobel1Full= np.array([])
edge_scharr1Full= np.array([])
edge_prewitt1Full= np.array([])
gaussian_img1Full= np.array([])
gaussian_img3Full= np.array([])
median_img1Full= np.array([])
variance_img1Full= np.array([])


for img2 in imagGreyArra:
    #CANNY EDGE
    edges = cv2.Canny(img2, 100,200)   #Image, min and max values
    edges1 = edges.reshape(-1)
    edges1Full= np.concatenate((edges1Full,edges1),axis=None)

    #ROBERTS EDGE
    edge_roberts = roberts(img2)
    edge_roberts1 = edge_roberts.reshape(-1)
    edge_roberts1Full= np.concatenate((edge_roberts1Full,edge_roberts1),axis=None)

    #SOBEL
    edge_sobel = sobel(img2)
    edge_sobel1 = edge_sobel.reshape(-1)
    edge_sobel1Full= np.concatenate((edge_sobel1Full,edge_sobel1),axis=None)
    #SCHARR
    edge_scharr = scharr(img2)
    edge_scharr1 = edge_scharr.reshape(-1)
    edge_scharr1Full= np.concatenate((edge_scharr1Full,edge_scharr1),axis=None)
    #PREWITT
    edge_prewitt = prewitt(img2)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    edge_prewitt1Full= np.concatenate((edge_prewitt1Full,edge_prewitt1),axis=None)
    #GAUSSIAN with sigma=3
    gaussian_img = nd.gaussian_filter(img2, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    gaussian_img1Full= np.concatenate((gaussian_img1Full,gaussian_img1),axis=None)
    #GAUSSIAN with sigma=7
    gaussian_img2 = nd.gaussian_filter(img2, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    gaussian_img3Full= np.concatenate((gaussian_img3Full,gaussian_img3),axis=None)
    #MEDIAN with sigma=3
    median_img = nd.median_filter(img2, size=3)
    median_img1 = median_img.reshape(-1)
    median_img1Full= np.concatenate((median_img1Full,median_img1),axis=None)
    #VARIANCE with size=3
    variance_img = nd.generic_filter(img2, np.var, size=3)
    variance_img1 = variance_img.reshape(-1)
    variance_img1Full= np.concatenate((variance_img1Full,variance_img1),axis=None)
    
df['Canny Edge'] = edges1Full #Add column to original dataframe
df['Roberts'] = edge_roberts1Full
df['Sobel'] = edge_sobel1Full
df['Scharr'] = edge_scharr1Full
df['Prewitt'] = edge_prewitt1Full
df['Gaussian s3'] = gaussian_img1Full
df['Gaussian s7'] = gaussian_img3Full
df['Median s3'] = median_img1Full
df['Variance s3'] = variance_img1Full  #Add column to original dataframe


In [ ]:
# In this run the feature below where excluded, uncomment then to explorer those features
# print(edges1Full.shape,edge_roberts1Full.shape)
# df['Canny Edge'] = edges1Full #Add column to original dataframe
# df['Roberts'] = edge_roberts1Full
# df['Sobel'] = edge_sobel1Full
# df['Scharr'] = edge_scharr1Full
# df['Prewitt'] = edge_prewitt1Full
# df['Gaussian s3'] = gaussian_img1Full
# df['Gaussian s7'] = gaussian_img3Full
# df['Median s3'] = median_img1Full
# df['Variance s3'] = variance_img1Full

In [15]:
df

,Original Image Hue,Original Image Saturation,Original Image Value,Original Image,Original Image Hist,Original Image Range,Original Image Gamma
0,62.0,14.0,255.0,249.0,255.0,0.0,0.0
1,62.0,16.0,255.0,249.0,255.0,0.0,0.0
2,62.0,15.0,255.0,249.0,255.0,0.0,0.0
3,62.0,14.0,255.0,249.0,255.0,0.0,0.0
4,62.0,14.0,251.0,245.0,255.0,0.0,0.0
...,...,...,...,...,...,...,...
1290370,35.0,11.0,249.0,247.0,251.0,0.0,0.0
1290371,51.0,7.0,243.0,241.0,251.0,0.0,255.0
1290372,48.0,5.0,255.0,254.0,255.0,0.0,0.0
1290373,36.0,5.0,255.0,254.0,254.0,0.0,0.0


In [16]:
locationLabels= "C:/Labels/RF_Clasified"
dataLabel= location+"/BluePrint40007.png"

In [ ]:
# Reading data with the labels
labeled_img1Full= np.array([])
for data in fileArra:
#     print(data)
    splitA=data.split("Clasification")[1]
    splitB=splitA.split("\\")[1]
    name= splitB.split(".")[0]
#     print(name)
    clasName=name+"_clasified.png"
    dataLabel=locationLabels+"/"+clasName
    print(dataLabel)
    #Now, add a column in the data frame for the Labels
    #For this, we need to import the labeled image
    labeled_img = cv2.imread(dataLabel)
    #Remember that you can load an image with partial labels 
    #But, drop the rows with unlabeled data

    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
    labeled_img1 = labeled_img.reshape(-1)
    labeled_img1Full=np.concatenate((labeled_img1Full,labeled_img1),axis=None)
df['Labels'] = labeled_img1Full
print(df.head())

In [18]:
######################################                
#########################################################

In [19]:
#########################################################

#Define the dependent variable that needs to be predicted (labels)
Y = df["Labels"].values

#Define the independent variables
X = df.drop(labels = ["Labels"], axis=1) 

#Split data into train and test to verify accuracy after fitting the model. 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=20)


# Import the model we are using
#For classification we use RandomForestClassifier.
#Here you can change between RandomForest or XGBoost. 

# from sklearn.ensemble import RandomForestClassifier
# Instantiate model with n number of decision trees
model = xgb.XGBClassifier()
# model = RandomForestClassifier(n_estimators = 200, random_state = 42,n_jobs=-1)

In [20]:
#SVM
# Train the Linear SVM to compare against Random Forest
#SVM will be slower than Random Forest. 
#Make sure to comment out Fetaure importances lines of code as it does not apply to SVM.
#from sklearn.svm import LinearSVC
#model = LinearSVC(max_iter=100)  #Default of 100 is not converging

# Train the model on training data
model.fit(X_train, y_train)

# verify number of trees used. If not defined above. 
#print('Number of Trees used : ', model.n_estimators)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:10:23] WARNING: D:\bld\xgboost-split_1631904903843\work\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
# this line is for RandomForest
# print('Number of Trees used : ', model.n_estimators)

In [22]:
#STEP 8: TESTING THE MODEL BY PREDICTING ON TEST DATA
#AND CALCULATE THE ACCURACY SCORE
#First test prediction on the training data itself. SHould be good. 
prediction_test_train = model.predict(X_train)

#Test prediction on testing data. 
prediction_test = model.predict(X_test)

#.predict just takes the .predict_proba output and changes everything 
#to 0 below a certain threshold (usually 0.5) respectively to 1 above that threshold.
#In this example we have 4 labels, so the probabilities will for each label stored separately. 
# 
#prediction_prob_test = model.predict_proba(X_test)

In [23]:
prediction_test_train

array([0., 0., 0., ..., 0., 0., 0.])

In [24]:
prediction_test

array([0., 0., 0., ..., 0., 0., 0.])

In [25]:

#Let us check the accuracy on test data
from sklearn import metrics
#Print the prediction accuracy

#First check the accuracy on training data. This will be higher than test data prediction accuracy.
print ("Accuracy on training data = ", metrics.accuracy_score(y_train, prediction_test_train))
#Check accuracy on test dataset. If this is too low compared to train it indicates overfitting on training data.
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_test))

Accuracy on training data =  0.9959998708385805
Accuracy =  0.99515644676935


In [26]:

#This part commented out for SVM testing. Uncomment for random forest. 
#One amazing feature of Random forest and XGBoost is that it provides us info on feature importances
# Get numerical feature importances
importances = list(model.feature_importances_)

#Let us print them into a nice format.

In [27]:
importances

[0.3153182,
 0.056979544,
 0.09045442,
 0.082824335,
 0.25068504,
 0.16164985,
 0.042088557]

In [28]:

feature_list = list(X.columns)
feature_imp = pd.Series(model.feature_importances_,index=feature_list).sort_values(ascending=False)
print(feature_imp)

Original Image Hue           0.315318
Original Image Hist          0.250685
Original Image Range         0.161650
Original Image Value         0.090454
Original Image               0.082824
Original Image Saturation    0.056980
Original Image Gamma         0.042089
dtype: float32


In [29]:
location = "C:/SavedModel"
dataSaveModel= location+"/RainRFModel08OCT2021_1108.jpg"

In [30]:
noContribution= [1,2,13,9,10,14,15,16,17,18,25,26,27,19] 

In [31]:
import pickle

#Save the trained model as pickle string to disk for future use
filename = "bM_XGBOOST_Explore_RainIRHSVMultiHistRange08OCT"
pickle.dump(model, open(filename, 'wb'))

#To test the model on future datasets
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(X)

# segmented = result.reshape((img2.shape))
# from matplotlib import pyplot as plt
# plt.imshow(segmented, cmap ='jet')
# plt.imsave('bM_RF_Explore_RainIRHSV.jpg', segmented, cmap ='jet')